## Colab Prep

Execute the following code cells to whenever you open/restart the notebook in Google Colab.

In [1]:
!pip install "polars[all]"

In [2]:
!wget https://github.com/WSU-DataScience/dsci_325_module_7_more_data_management_in_python/raw/main/sample_data.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!unzip ./sample_data.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!pip install more_polars

# Joining Tables with `polars`

In [5]:
import polars as pl
pl.Config.with_columns_kwargs = True

## Example

The following tables were taken from the Wikipedia page for joins and will provide a small but sufficiently complicated example for introducing joins.

In [6]:
dept = pl.read_csv("./sample_data/department.csv")
dept

DeptID,DeptName
i64,str
31,"""Sales"""
33,"""Engineering"""
34,"""Clerical"""
35,"""Marketing"""


In [7]:
empl = pl.read_csv("./sample_data/employee.csv")
empl

LastName,EmployeeID,DeptID
str,i64,i64
"""Rafferty""",102,31
"""Jones""",105,33
"""Heisenberg""",106,33
"""Robinson""",108,34
"""Smith""",110,34
"""Williams""",111,null


In [8]:
salary = pl.read_csv("./sample_data/salary.csv")
salary

EmployeeID,Year,Salary
i64,i64,i64
102,2020,52000
102,2021,53000
102,2022,54000
105,2020,73000
105,2021,74000
…,…,…
110,2021,59000
110,2022,61000
111,2020,72000


## LEFT JOIN

In [9]:
empl.join(dept, on='DeptID', how='left')

LastName,EmployeeID,DeptID,DeptName
str,i64,i64,str
"""Rafferty""",102,31,"""Sales"""
"""Jones""",105,33,"""Engineering"""
"""Heisenberg""",106,33,"""Engineering"""
"""Robinson""",108,34,"""Clerical"""
"""Smith""",110,34,"""Clerical"""
"""Williams""",111,null,null


## When the index names differ

In [10]:
empl2 = empl.rename({'DeptID':'dept_id'})
empl2

LastName,EmployeeID,dept_id
str,i64,i64
"""Rafferty""",102,31
"""Jones""",105,33
"""Heisenberg""",106,33
"""Robinson""",108,34
"""Smith""",110,34
"""Williams""",111,null


In [11]:
empl2.join(dept, left_on='dept_id', right_on='DeptID', how='left')

LastName,EmployeeID,dept_id,DeptName
str,i64,i64,str
"""Rafferty""",102,31,"""Sales"""
"""Jones""",105,33,"""Engineering"""
"""Heisenberg""",106,33,"""Engineering"""
"""Robinson""",108,34,"""Clerical"""
"""Smith""",110,34,"""Clerical"""
"""Williams""",111,null,null


## RIGHT JOIN

In [12]:
empl2.join(dept, left_on='dept_id', right_on='DeptID', how='right')

LastName,EmployeeID,DeptID,DeptName
str,i64,i64,str
"""Rafferty""",102,31,"""Sales"""
"""Jones""",105,33,"""Engineering"""
"""Heisenberg""",106,33,"""Engineering"""
"""Robinson""",108,34,"""Clerical"""
"""Smith""",110,34,"""Clerical"""
null,null,35,"""Marketing"""


## Full Outer Join

In [13]:
empl.join(dept, on='DeptID', how='full')

LastName,EmployeeID,DeptID,DeptID_right,DeptName
str,i64,i64,i64,str
"""Rafferty""",102,31,31,"""Sales"""
"""Jones""",105,33,33,"""Engineering"""
"""Heisenberg""",106,33,33,"""Engineering"""
"""Robinson""",108,34,34,"""Clerical"""
"""Smith""",110,34,34,"""Clerical"""
"""Williams""",111,null,null,null
null,null,null,35,"""Marketing"""


## Inner Join

In [14]:
empl.join(dept, on='DeptID', how='inner')

LastName,EmployeeID,DeptID,DeptName
str,i64,i64,str
"""Rafferty""",102,31,"""Sales"""
"""Jones""",105,33,"""Engineering"""
"""Heisenberg""",106,33,"""Engineering"""
"""Robinson""",108,34,"""Clerical"""
"""Smith""",110,34,"""Clerical"""


## <font color="red"> Exercise 7.3.1 </font>

**Tasks**

1. Use help on `empl.join` and identify any additional join types offered by `polars`, and
2. Write a sentence or two providing an example that might require each join type.

In [45]:
?empl.join

Signature:
empl.join(
    other: 'DataFrame',
    on: 'str | Expr | Sequence[str | Expr] | None' = None,
    how: 'JoinStrategy' = 'inner',
    *,
    left_on: 'str | Expr | Sequence[str | Expr] | None' = None,
    right_on: 'str | Expr | Sequence[str | Expr] | None' = None,
    suffix: 'str' = '_right',
    validate: 'JoinValidation' = 'm:m',
    join_nulls: 'bool' = False,
    coalesce: 'bool | None' = None,
) -> 'DataFrame'
Docstring:
Join in SQL-like fashion.

Parameters
----------
other
    DataFrame to join with.
on
    Name(s) of the join columns in both DataFrames.
how : {'inner', 'left', 'right', 'full', 'semi', 'anti', 'cross'}
    Join strategy.

    * *inner*
        Returns rows that have matching values in both tables
    * *left*
        Returns all rows from the left table, and the matched rows from the
        right table
    * *right*
        Returns all rows from the right table, and the matched rows from the
        left table
    * *full*
        Returns all rows w

<font color="orange">
Your answers here

1. Semi - would be used to return rows from the left table that have a match in the right table; This would be useful if you only wanted to join matching values in a table
2. Anti - Returns rows from the left table that have no match in the right table; This would be useful to know what variables do not have any matches in the other table.
</font>

## Chaining multiple joins

To join more than two tables, simply dot-chain each join onto the first table.

```{python}
(table1
 .join(table2, ...)
 .join(table3, ...)
)
```

### Example - Average salary by department

Suppose we want to compute the average salary for each department, sorted from largest to smallest.  Before performing the query, make sure you understand the relationships between tables.

<img src="https://github.com/WSU-DataScience/dsci_325_module_7_more_data_management_in_python/raw/main/img/table_relationship.png" width="800">


#### Edit 1 - Joining the salary and employee tables

**Notes.**

1. We can't perform the aggregation until *after* joining the departments and salaries.
2. Since we are primarily interested in aggregating salaries, we need to use a *left* join with the salary table on the left.
3. To illustrate the coding process, we will show each successive edit in a separate cell.  In practice, all the code would be developed in one cell, but rerun to check the results at each step.

In [16]:
(salary
 .join(empl, on='EmployeeID', how='left')
).head()

EmployeeID,Year,Salary,LastName,DeptID
i64,i64,i64,str,i64
102,2020,52000,"""Rafferty""",31
102,2021,53000,"""Rafferty""",31
102,2022,54000,"""Rafferty""",31
105,2020,73000,"""Jones""",33
105,2021,74000,"""Jones""",33


#### Edit 2  - Joining on the departments

In [17]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
).head()

EmployeeID,Year,Salary,LastName,DeptID,DeptName
i64,i64,i64,str,i64,str
102,2020,52000,"""Rafferty""",31,"""Sales"""
102,2021,53000,"""Rafferty""",31,"""Sales"""
102,2022,54000,"""Rafferty""",31,"""Sales"""
105,2020,73000,"""Jones""",33,"""Engineering"""
105,2021,74000,"""Jones""",33,"""Engineering"""


#### Edit 3  - Group and aggregate

In [18]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
 .group_by('DeptName')
 .agg([pl.col('Salary').mean().alias('Average Salary'),
      ])
)

DeptName,Average Salary
str,f64
"""Engineering""",79000.0
"""Sales""",53000.0
null,73000.0
"""Clerical""",77000.0


#### Edit 4  - Sort the results from largest to smallest

In [19]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
 .group_by('DeptName')
 .agg([pl.col('Salary').mean().alias('Average Salary'),
      ])
 .sort('Average Salary', descending=True)
)

DeptName,Average Salary
str,f64
"""Engineering""",79000.0
"""Clerical""",77000.0
null,73000.0
"""Sales""",53000.0


## Example 2 - Joining Batting to People

As an example of a joining on more than 1 column, consider the following task.

**Task:** Create a table with the total runs score for each team in 2009.  Include the teams proper name and the name of their home park.

In [20]:
batting = pl.read_csv("./sample_data/baseball/core/Batting.csv")
batting.head()

playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64
"""abercda01""",1871,1,"""TRO""","""NA""",1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""","""NA""",25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0
"""allisar01""",1871,1,"""CL1""","""NA""",29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1
"""allisdo01""",1871,1,"""WS3""","""NA""",27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0
"""ansonca01""",1871,1,"""RC1""","""NA""",25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0


In [21]:
teams = pl.read_csv("./sample_data/baseball/core/Teams.csv")
teams.head()

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
i64,str,str,str,str,i64,i64,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,str,i64,i64,str,str,str
1871,"""NA""","""BS1""","""BNA""",null,3,31,null,20,10,null,null,"""N""",null,401,1372,426,70,37,3,60,19,73,16,null,null,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,"""Boston Red Stockings""","""South End Grounds I""",null,103,98,"""BOS""","""BS1""","""BS1"""
1871,"""NA""","""CH1""","""CNA""",null,2,28,null,19,9,null,null,"""N""",null,302,1196,323,52,21,10,60,22,69,21,null,null,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,"""Chicago White Stockings""","""Union Base-Ball Grounds""",null,104,102,"""CHI""","""CH1""","""CH1"""
1871,"""NA""","""CL1""","""CFC""",null,8,29,null,10,19,null,null,"""N""",null,249,1186,328,35,40,7,26,25,18,8,null,null,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,"""Cleveland Forest Citys""","""National Association Grounds""",null,96,100,"""CLE""","""CL1""","""CL1"""
1871,"""NA""","""FW1""","""KEK""",null,7,19,null,7,12,null,null,"""N""",null,137,746,178,19,8,2,33,9,16,4,null,null,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,"""Fort Wayne Kekiongas""","""Hamilton Field""",null,101,107,"""KEK""","""FW1""","""FW1"""
1871,"""NA""","""NY2""","""NNA""",null,5,33,null,16,17,null,null,"""N""",null,302,1404,403,43,21,1,33,15,46,15,null,null,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.84,"""New York Mutuals""","""Union Grounds (Brooklyn)""",null,90,88,"""NYU""","""NY2""","""NY2"""


## Things we need from `Batting.csv`

* Year
* Runs
* Team

In [22]:
batting.head(2)

playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64
"""abercda01""",1871,1,"""TRO""","""NA""",1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""","""NA""",25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0


#### Selecting down to the necessary columns

In [23]:
batting_select = (batting
                  .select(['teamID',
                           'yearID',
                           'R'])
                 )
batting_select.head()

teamID,yearID,R
str,i64,i64
"""TRO""",1871,0
"""RC1""",1871,30
"""CL1""",1871,28
"""WS3""",1871,28
"""RC1""",1871,29


In [24]:
batting_select.shape

(107429, 3)

## Things we need from `Teams.csv`

* Year
* Runs
* Team

In [25]:
teams.head(2)

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
i64,str,str,str,str,i64,i64,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,str,i64,i64,str,str,str
1871,"""NA""","""BS1""","""BNA""",null,3,31,null,20,10,null,null,"""N""",null,401,1372,426,70,37,3,60,19,73,16,null,null,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,"""Boston Red Stockings""","""South End Grounds I""",null,103,98,"""BOS""","""BS1""","""BS1"""
1871,"""NA""","""CH1""","""CNA""",null,2,28,null,19,9,null,null,"""N""",null,302,1196,323,52,21,10,60,22,69,21,null,null,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,"""Chicago White Stockings""","""Union Base-Ball Grounds""",null,104,102,"""CHI""","""CH1""","""CH1"""


#### Selecting down to the necessary columns

In [26]:
teams_select = (teams.select(['yearID',
                              'teamID',
                              'name',
                              'park',
                             ]))
teams_select.head()

yearID,teamID,name,park
i64,str,str,str
1871,"""BS1""","""Boston Red Stockings""","""South End Grounds I"""
1871,"""CH1""","""Chicago White Stockings""","""Union Base-Ball Grounds"""
1871,"""CL1""","""Cleveland Forest Citys""","""National Association Grounds"""
1871,"""FW1""","""Fort Wayne Kekiongas""","""Hamilton Field"""
1871,"""NY2""","""New York Mutuals""","""Union Grounds (Brooklyn)"""


In [27]:
teams_select.shape

(2925, 4)

## How can we match these columns?

We will need to match on both `teamID` and `yearID` because

1. Team names might change
2. The teams park depends on the year.

## Determining the correct join.

Facts of the task:

* We want batting statistics for all players in 2009
* We want only teams names and parks for teams that played that year.

**Question:** Which join? 
**Answer:** `batting >> left_join(teams)` on both year and 

## How to join on multiple columns

Pass a list of columns names to `on`

In [28]:
df_joined = batting_select.join(teams_select, on=['teamID', 'yearID'], how='left')
df_joined.head()

teamID,yearID,R,name,park
str,i64,i64,str,str
"""TRO""",1871,0,"""Troy Haymakers""","""Haymakers' Grounds"""
"""RC1""",1871,30,"""Rockford Forest Citys""","""Agricultural Society Fair Grou…"
"""CL1""",1871,28,"""Cleveland Forest Citys""","""National Association Grounds"""
"""WS3""",1871,28,"""Washington Olympics""","""Olympics Grounds"""
"""RC1""",1871,29,"""Rockford Forest Citys""","""Agricultural Society Fair Grou…"


In [29]:
df_joined.shape

(107429, 5)

#### Filter, group by, and aggregate to get the desired result.

In [30]:
runs_per_team09 = (df_joined
                   .filter(pl.col('yearID') == 2009)
                   .group_by([pl.col('name'), pl.col('park')])
                   .agg(pl.col('R').sum().alias('total_runs')))
runs_per_team09.head()

name,park,total_runs
str,str,i64
"""Milwaukee Brewers""","""Miller Park""",785
"""Baltimore Orioles""","""Oriole Park at Camden Yards""",741
"""Cincinnati Reds""","""Great American Ball Park""",673
"""Chicago White Sox""","""U.S. Cellular Field""",724
"""Arizona Diamondbacks""","""Chase Field""",720


In [31]:
runs_per_team09.shape

(30, 3)

## <font color="red"> Exercise 7.3.2 </font>

Determine all the players that have hit more than 30 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the `Batting.csv`, `People.csv`, and `Teams.csv` files.  To get credit for this exercise, use the join methods presented above.

In [47]:
Batting = pl.read_csv("./sample_data/baseball/core/Batting.csv")
Batting

playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64
"""abercda01""",1871,1,"""TRO""","""NA""",1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""","""NA""",25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0
"""allisar01""",1871,1,"""CL1""","""NA""",29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1
"""allisdo01""",1871,1,"""WS3""","""NA""",27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0
"""ansonca01""",1871,1,"""RC1""","""NA""",25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zimmejo02""",2019,1,"""DET""","""AL""",23,2,0,0,0,0,0,0,0,0,0,2,"""0""","""0""","""0""","""0""",0
"""zimmeky01""",2019,1,"""KCA""","""AL""",15,0,0,0,0,0,0,0,0,0,0,0,"""0""","""0""","""0""","""0""",0
"""zimmery01""",2019,1,"""WAS""","""NL""",52,171,20,44,9,0,6,27,0,0,17,39,"""0""","""0""","""0""","""2""",4


In [48]:
People = pl.read_csv("./sample_data/baseball/core/People.csv")
People

playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
str,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str
"""aardsda01""",1981,12,27,"""USA""","""CO""","""Denver""",null,null,null,null,null,null,"""David""","""Aardsma""","""David Allan""",215,75,"""R""","""R""","""2004-04-06""","""2015-08-23""","""aardd001""","""aardsda01"""
"""aaronha01""",1934,2,5,"""USA""","""AL""","""Mobile""",null,null,null,null,null,null,"""Hank""","""Aaron""","""Henry Louis""",180,72,"""R""","""R""","""1954-04-13""","""1976-10-03""","""aaroh101""","""aaronha01"""
"""aaronto01""",1939,8,5,"""USA""","""AL""","""Mobile""",1984,8,16,"""USA""","""GA""","""Atlanta""","""Tommie""","""Aaron""","""Tommie Lee""",190,75,"""R""","""R""","""1962-04-10""","""1971-09-26""","""aarot101""","""aaronto01"""
"""aasedo01""",1954,9,8,"""USA""","""CA""","""Orange""",null,null,null,null,null,null,"""Don""","""Aase""","""Donald William""",190,75,"""R""","""R""","""1977-07-26""","""1990-10-03""","""aased001""","""aasedo01"""
"""abadan01""",1972,8,25,"""USA""","""FL""","""Palm Beach""",null,null,null,null,null,null,"""Andy""","""Abad""","""Fausto Andres""",184,73,"""L""","""L""","""2001-09-10""","""2006-04-13""","""abada001""","""abadan01"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zupofr01""",1939,8,29,"""USA""","""CA""","""San Francisco""",2005,3,25,"""USA""","""CA""","""Burlingame""","""Frank""","""Zupo""","""Frank Joseph""",182,71,"""L""","""R""","""1957-07-01""","""1961-05-09""","""zupof101""","""zupofr01"""
"""zuvelpa01""",1958,10,31,"""USA""","""CA""","""San Mateo""",null,null,null,null,null,null,"""Paul""","""Zuvella""","""Paul""",173,72,"""R""","""R""","""1982-09-04""","""1991-05-02""","""zuvep001""","""zuvelpa01"""
"""zuverge01""",1924,8,20,"""USA""","""MI""","""Holland""",2014,9,8,"""USA""","""AZ""","""Tempe""","""George""","""Zuverink""","""George""",195,76,"""R""","""R""","""1951-04-21""","""1959-06-15""","""zuveg101""","""zuverge01"""


In [50]:
Team = pl.read_csv("./sample_data/baseball/core/Teams.csv")
Team

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
i64,str,str,str,str,i64,i64,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,str,i64,i64,str,str,str
1871,"""NA""","""BS1""","""BNA""",null,3,31,null,20,10,null,null,"""N""",null,401,1372,426,70,37,3,60,19,73,16,null,null,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,"""Boston Red Stockings""","""South End Grounds I""",null,103,98,"""BOS""","""BS1""","""BS1"""
1871,"""NA""","""CH1""","""CNA""",null,2,28,null,19,9,null,null,"""N""",null,302,1196,323,52,21,10,60,22,69,21,null,null,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,"""Chicago White Stockings""","""Union Base-Ball Grounds""",null,104,102,"""CHI""","""CH1""","""CH1"""
1871,"""NA""","""CL1""","""CFC""",null,8,29,null,10,19,null,null,"""N""",null,249,1186,328,35,40,7,26,25,18,8,null,null,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,"""Cleveland Forest Citys""","""National Association Grounds""",null,96,100,"""CLE""","""CL1""","""CL1"""
1871,"""NA""","""FW1""","""KEK""",null,7,19,null,7,12,null,null,"""N""",null,137,746,178,19,8,2,33,9,16,4,null,null,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,"""Fort Wayne Kekiongas""","""Hamilton Field""",null,101,107,"""KEK""","""FW1""","""FW1"""
1871,"""NA""","""NY2""","""NNA""",null,5,33,null,16,17,null,null,"""N""",null,302,1404,403,43,21,1,33,15,46,15,null,null,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.84,"""New York Mutuals""","""Union Grounds (Brooklyn)""",null,90,88,"""NYU""","""NY2""","""NY2"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019,"""NL""","""SLN""","""STL""","""C""",1,162,"""81""",91,71,"""Y""","""N""","""N""","""N""",764,5449,1336,246,24,210,561,1420,116,29,"""76""","""39""",662,609,3.8,1,14,52,4332,1284,191,545,1399,66,168,0.989,"""St. Louis Cardinals""","""Busch Stadium III""","""3480393""",98,97,"""STL""","""SLN""","""SLN"""
2019,"""AL""","""TBA""","""TBD""","""E""",2,162,"""81""",96,66,"""N""","""Y""","""N""","""N""",769,5628,1427,291,29,217,542,1493,94,37,"""73""","""34""",656,598,3.65,0,12,46,4423,1274,181,453,1621,87,126,0.985,"""Tampa Bay Rays""","""Tropicana Field""","""1178735""",97,96,"""TBR""","""TBA""","""TBA"""
2019,"""AL""","""TEX""","""TEX""","""W""",3,162,"""81""",78,84,"""N""","""N""","""N""","""N""",810,5540,1374,296,24,223,534,1578,131,38,"""67""","""44""",878,808,5.06,4,9,33,4314,1515,241,583,1379,105,143,0.982,"""Texas Rangers""","""Globe Life Park in Arlington""","""2132994""",111,112,"""TEX""","""TEX""","""TEX"""


In [51]:
batting_select = (batting
                  .select(['teamID',
                           'playerID',
                           'HR'])
                 )
batting_select.head()

teamID,playerID,HR
str,str,i64
"""TRO""","""abercda01""",0
"""RC1""","""addybo01""",0
"""CL1""","""allisar01""",0
"""WS3""","""allisdo01""",2
"""RC1""","""ansonca01""",0


In [53]:
teams_select = (teams.select(['name',
                              'teamID',
                             ]))
teams_select.head()

name,teamID
str,str
"""Boston Red Stockings""","""BS1"""
"""Chicago White Stockings""","""CH1"""
"""Cleveland Forest Citys""","""CL1"""
"""Fort Wayne Kekiongas""","""FW1"""
"""New York Mutuals""","""NY2"""


In [55]:
People_select = (People.select(['playerID',
                              'nameGiven',
                             ]))
People_select.head()

playerID,nameGiven
str,str
"""aardsda01""","""David Allan"""
"""aaronha01""","""Henry Louis"""
"""aaronto01""","""Tommie Lee"""
"""aasedo01""","""Donald William"""
"""abadan01""","""Fausto Andres"""


In [58]:
df_joined = batting_select.join(teams_select, on=['teamID'], how='left')
df_joined.head()

teamID,playerID,HR,name
str,str,i64,str
"""TRO""","""abercda01""",0,"""Troy Haymakers"""
"""TRO""","""abercda01""",0,"""Troy Haymakers"""
"""RC1""","""addybo01""",0,"""Rockford Forest Citys"""
"""CL1""","""allisar01""",0,"""Cleveland Forest Citys"""
"""CL1""","""allisar01""",0,"""Cleveland Forest Citys"""


In [59]:
df_joined2 = df_joined.join(People_select, on=['playerID'], how='left')
df_joined2.head()

teamID,playerID,HR,name,nameGiven
str,str,i64,str,str
"""TRO""","""abercda01""",0,"""Troy Haymakers""","""Francis Patterson"""
"""TRO""","""abercda01""",0,"""Troy Haymakers""","""Francis Patterson"""
"""RC1""","""addybo01""",0,"""Rockford Forest Citys""","""Robert Edward"""
"""CL1""","""allisar01""",0,"""Cleveland Forest Citys""","""Arthur Algernon"""
"""CL1""","""allisar01""",0,"""Cleveland Forest Citys""","""Arthur Algernon"""


In [60]:
over_thirty_HR = (df_joined2
                   .filter(pl.col('HR') > 30)
                 )
over_thirty_HR.head()

teamID,playerID,HR,name,nameGiven
str,str,i64,str,str
"""NYA""","""ruthba01""",54,"""New York Highlanders""","""George Herman"""
"""NYA""","""ruthba01""",54,"""New York Highlanders""","""George Herman"""
"""NYA""","""ruthba01""",54,"""New York Highlanders""","""George Herman"""
"""NYA""","""ruthba01""",54,"""New York Highlanders""","""George Herman"""
"""NYA""","""ruthba01""",54,"""New York Highlanders""","""George Herman"""


In [61]:
df_unique = over_thirty_HR.unique()
df_unique.head()

teamID,playerID,HR,name,nameGiven
str,str,i64,str,str
"""NYA""","""ruthba01""",47,"""New York Yankees""","""George Herman"""
"""NYA""","""nettlgr01""",37,"""New York Yankees""","""Graig"""
"""TOR""","""delgaca01""",32,"""Toronto Blue Jays""","""Carlos Juan"""
"""TOR""","""wellsve01""",32,"""Toronto Blue Jays""","""Vernon M."""
"""CHN""","""sosasa01""",35,"""Chicago Cubs""","""Samuel Peralta"""


In [62]:
HR_select = (df_unique.select(['name',
                              'nameGiven',
                             ]))
HR_select.head()

name,nameGiven
str,str
"""New York Yankees""","""George Herman"""
"""New York Yankees""","""Graig"""
"""Toronto Blue Jays""","""Carlos Juan"""
"""Toronto Blue Jays""","""Vernon M."""
"""Chicago Cubs""","""Samuel Peralta"""


## Joins are expensive!

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `inner_join` performs `min(len(A), len(B))` comparisons
* `outer_join` performs `len(A)*len(B)` row comparisons.
* `left_join` and `right_join` are somewhere inbetween (depending on how we treat multiples).

## Outer joins (can) produce LARGE tables.

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `outer_join` might produce `len(A)*len(B)` rows (worst case)
* `left_join` and `right_join` are similar if we don't drop multiples.

### Example - Incorrect join of `batting` and `teams`

Suppose that, when joining the batting and teams table, we forget to include the `yearID` as a key

In [33]:
batting.shape

(107429, 22)

In [34]:
teams.shape

(2925, 48)

In [35]:
BAD = batting.join(teams, on='teamID', how='full')

In [36]:
# the output EXPLODED!!!1!!one!
BAD.shape

(8699563, 70)

#### Last example 

In [37]:
len(batting_select)*len(teams_select)

314229825

## The joining mantra: Filter/aggregate THEN join

By filtering and/or aggregating before joining, you will

* reduce the amount of work
* protect against really large output

#### Filter and aggregate to total runs per team per year

In [38]:
runs = (batting_select
        .filter(pl.col('yearID') == 2009)
        .group_by(pl.col('teamID'))
        .agg(pl.col('R').sum().alias('total_runs')))

runs.head()

teamID,total_runs
str,i64
"""BOS""",872
"""SEA""",640
"""SFN""",657
"""LAA""",883
"""ATL""",735


#### Filter down to 2009 teams

In [39]:
teams09 = (teams_select
           .filter(pl.col('yearID') == 2009)
          )
teams09.head()

yearID,teamID,name,park
i64,str,str,str
2009,"""ARI""","""Arizona Diamondbacks""","""Chase Field"""
2009,"""ATL""","""Atlanta Braves""","""Turner Field"""
2009,"""BAL""","""Baltimore Orioles""","""Oriole Park at Camden Yards"""
2009,"""BOS""","""Boston Red Sox""","""Fenway Park II"""
2009,"""CHA""","""Chicago White Sox""","""U.S. Cellular Field"""


In [40]:
len(runs)*len(teams09)

900

In [41]:
runs_per_team09_new = (runs 
                       .join(teams09, on='teamID', how='left')
                       .drop('teamID')
                      )
runs_per_team09_new.head()

total_runs,yearID,name,park
i64,i64,str,str
872,2009,"""Boston Red Sox""","""Fenway Park II"""
640,2009,"""Seattle Mariners""","""Safeco Field"""
657,2009,"""San Francisco Giants""","""AT&T Park"""
883,2009,"""Los Angeles Angels of Anaheim""","""Angel Stadium"""
735,2009,"""Atlanta Braves""","""Turner Field"""


In [42]:
runs_per_team09_new.shape

(30, 4)

## Don't worry about speed until it matters!

* I don't emphasize speed most of the time.
* In this case, either approach was nearly instant.
* Still ... `join`s are *so dangerous* that I make an exception here!

In [43]:
# Your code here